In [1]:
import os
import sys
import copy
import pickle
import concurrent.futures
import numpy as np
import pandas as pd

import cantera as ct
import rmgpy.chemkin

sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/kinetics')
import kineticfun


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(reaction_list)} reactions and {len(species_list)} species')

1556030143.py:7 <module> INFO Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


1822 reactions and 110 species


In [3]:
base_delay = 0.14857535

In [4]:
ignition_delay_rxns = base_delay * np.ones(len(reaction_list))
ignition_delay_sp = base_delay * np.ones(len(species_list))

In [5]:
# load individual ignition delay files
results_dir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/models/reactions'
for i in range(0, len(reaction_list)):
    result_file = os.path.join(results_dir, f'delay_{i:04}.txt')
    if not os.path.exists(result_file):
        continue
    with open(result_file, 'r') as f:
        result = f.read()
        try:
            ignition_delay_rxns[i] = result
        except ValueError:
            continue
            
results_dir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/models/species'
for i in range(0, len(species_list)):
    result_file = os.path.join(results_dir, f'delay_{i:04}.txt')
    if not os.path.exists(result_file):
        continue
    with open(result_file, 'r') as f:
        result = f.read()
        try:
            ignition_delay_sp[i] = result
        except ValueError:
            continue

rxn_df = pd.DataFrame(ignition_delay_rxns, columns=['delay'])
rxn_df.to_csv(os.path.join(results_dir, 'reaction_delays.csv'))
species_df = pd.DataFrame(ignition_delay_sp, columns=['delay'])
species_df.to_csv(os.path.join(results_dir, 'species_delays.csv'))

# results_dir = '.'
# rxn_df = pd.read_csv(os.path.join(results_dir, 'reaction_delays.csv'))
# species_df = pd.read_csv(os.path.join(results_dir, 'species_delays.csv'))
# ignition_delay_rxns = rxn_df['delay'].values
# ignition_delay_sp = species_df['delay'].values

In [6]:
ignition_delay_rxns[288]

0.12905077586632863

In [7]:
np.average(ignition_delay_rxns)

0.14856756683843467

In [8]:
ignition_delay_sp[85]

0.17446256129346216

In [9]:
np.average(ignition_delay_sp)

0.149834571732

In [10]:
# sort the reaction list by difference in delay
order_rxn = np.arange(0, len(ignition_delay_rxns))
fs_diff_rxn = np.abs(ignition_delay_rxns - base_delay)
most_sensitive_rxn = [x for _, x in sorted(zip(fs_diff_rxn, order_rxn))][::-1]


order_sp = np.arange(0, len(ignition_delay_sp))
fs_diff = np.abs(ignition_delay_sp - base_delay)
most_sensitive_sp = [x for _, x in sorted(zip(fs_diff, order_sp))][::-1]

In [12]:
most_sensitive_sp

[59,
 58,
 73,
 72,
 74,
 85,
 46,
 5,
 32,
 21,
 11,
 57,
 37,
 4,
 56,
 45,
 17,
 60,
 87,
 50,
 38,
 13,
 43,
 63,
 12,
 70,
 20,
 39,
 29,
 9,
 69,
 25,
 18,
 22,
 30,
 62,
 27,
 36,
 68,
 15,
 61,
 31,
 42,
 35,
 84,
 23,
 14,
 66,
 19,
 105,
 47,
 10,
 103,
 90,
 54,
 65,
 51,
 55,
 16,
 28,
 26,
 64,
 33,
 24,
 102,
 76,
 48,
 67,
 109,
 41,
 95,
 52,
 40,
 99,
 104,
 75,
 8,
 107,
 83,
 92,
 53,
 34,
 49,
 98,
 71,
 80,
 77,
 7,
 100,
 78,
 97,
 88,
 81,
 6,
 101,
 94,
 79,
 106,
 86,
 44,
 108,
 91,
 1,
 82,
 3,
 2,
 0,
 96,
 93,
 89]

In [21]:
# save the reaction sensitivity results
len(fs_diff_rxn)
np.save('reaction_ignition_delay.npy', fs_diff_rxn)

In [13]:
# save the species sensitivity results
len(fs_diff)
np.save('species_ignition_delay.npy', fs_diff)

In [11]:
species_list[0].thermo

NASA(polynomials=[NASAPolynomial(coeffs=[3.61263,-0.00100893,2.49898e-06,-1.43375e-09,2.58634e-13,-1051.1,2.65271], Tmin=(100,'K'), Tmax=(1817.05,'K')), NASAPolynomial(coeffs=[2.97593,0.00164137,-7.19704e-07,1.25374e-10,-7.915e-15,-1025.86,5.53741], Tmin=(1817.05,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: BurkeH2O2""")

In [12]:
# print the top species
print('Rank\tIndex\tSource\t\tSpecies')
for i in range(0, 50):
    index = most_sensitive_sp[i]
    source = 'library'
    if 'library' not in species_list[index].thermo.comment.lower():
        source = 'GAV    '
#     print(i, '\t', index, '\t', source, '\t', np.round(ignition_delay_sp[index] - base_delay, 6),'\t', species_list[index])
    print(i, '\t', index, '\t', source, '\t', species_list[index].smiles)

Rank	Index	Source		Species
0 	 59 	 library 	 CCC(C)O[O]
1 	 58 	 library 	 CCCCO[O]
2 	 73 	 library 	 CC(CCOO)O[O]
3 	 72 	 library 	 [CH2]CC(C)OO
4 	 74 	 library 	 CC(CCO[O])OO
5 	 85 	 GAV     	 CC(CCOO)OO
6 	 46 	 library 	 CC(=O)COO
7 	 5 	 library 	 [O][O]
8 	 32 	 library 	 C[CH2]
9 	 21 	 library 	 [CH3]
10 	 11 	 library 	 O
11 	 57 	 library 	 C[CH]CC
12 	 37 	 library 	 COO
13 	 4 	 library 	 CCCC
14 	 56 	 library 	 [CH2]CCC
15 	 45 	 library 	 CC(=O)CO[O]
16 	 17 	 library 	 [H]
17 	 60 	 library 	 CCC(C)OO
18 	 87 	 library 	 [CH2]C(=O)COO
19 	 50 	 library 	 [CH2]C=C
20 	 38 	 library 	 CCO[O]
21 	 13 	 library 	 C
22 	 43 	 library 	 [CH2]C(C)=O
23 	 63 	 library 	 C=C[CH]C
24 	 12 	 library 	 C=O
25 	 70 	 library 	 C[CH]CCOO
26 	 20 	 library 	 OO
27 	 39 	 library 	 CCOO
28 	 29 	 library 	 CO
29 	 9 	 library 	 [C-]#[O+]
30 	 69 	 library 	 CCC(C)[O]
31 	 25 	 library 	 [CH]=C
32 	 18 	 library 	 [OH]
33 	 22 	 library 	 [CH]=O
34 	 30 	 library 	 C[O]
35 	 62 	 l

In [13]:
# print the top reactions
print('Rank\tIndex\tFamily\t\tReaction')
for i in range(0, 50):

    index = most_sensitive_rxn[i]
#     print(index, '\t', reaction_list[index])
    try:
#         if reaction_list[index].family != 'H_Abstraction' and \
#             reaction_list[index].family != 'Disproportionation' and \
#             reaction_list[index].family != 'intra_H_migration' and \
#             reaction_list[index].family != 'R_Addition_MultipleBond':
#             continue
        print(i,'\t', index, '\t', reaction_list[index].family, '\t', reaction_list[index], '\t')
    except AttributeError:
        continue
        print(i,'\t', index, '\t', 'PDEP', '\t', reaction_list[index], '\t')

Rank	Index	Family		Reaction
0 	 288 	 CurranPentane 	 OH(15) + butane(1) <=> H2O(8) + PC4H9(182) 	
1 	 247 	 CurranPentane 	 OH(15) + butane(1) <=> H2O(8) + SC4H9(183) 	
2 	 279 	 CurranPentane 	 SC4H9O2(186) <=> HO2(16) + C4H8-2(189) 	
3 	 323 	 CurranPentane 	 O2(2) + C4H8OOH1-3(219) <=> C4H8OOH1-3O2(225) 	
4 	 281 	 CurranPentane 	 O2(2) + C4H8OOH2-4(223) <=> C4H8OOH2-4O2(229) 	
5 	 316 	 CurranPentane 	 PC4H9O2(184) <=> HO2(16) + C4H8-1(188) 	
6 	 249 	 CurranPentane 	 HO2(16) + butane(1) <=> H2O2(17) + SC4H9(183) 	
7 	 449 	 CurranPentane 	 C4H8OOH1-3(219) <=> OH(15) + C4H8O1-3(214) 	
8 	 317 	 CurranPentane 	 SC4H9O2(186) <=> HO2(16) + C4H8-1(188) 	
9 	 326 	 CurranPentane 	 C4H8OOH1-3O2(225) <=> C4H71-1,3OOH(238) 	
10 	 290 	 CurranPentane 	 HO2(16) + butane(1) <=> H2O2(17) + PC4H9(182) 	
11 	 282 	 CurranPentane 	 C4H8OOH2-4O2(229) <=> C4H72-2,4OOH(244) 	
12 	 1701 	 CurranPentane 	 SC4H9O2(186) + butane(1) <=> SC4H9(183) + SC4H9O2H(187) 	
13 	 354 	 FFCM1(-) 	 HO2(16) + CH3CHO

In [20]:
covariance_file = '/work/westgroup/harris.se/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)

In [26]:
# sort the reaction list by uncertainty * sensitivity
order_rxn_importance = np.arange(0, len(ignition_delay_rxns))
delay_diff = np.multiply(np.abs(ignition_delay_rxns - base_delay), np.diagonal(Sigma_k))
most_important_rxn = [x for _, x in sorted(zip(delay_diff, order_rxn_importance))][::-1]


In [27]:
# print the top reactions by importance: uncertainty*sensitivity
print('Rank\tIndex\tFamily\t\tReaction')
for i in range(0, 50):

    index = most_important_rxn[i]
#     print(index, '\t', reaction_list[index])
    try:
#         if reaction_list[index].family != 'H_Abstraction' and \
#             reaction_list[index].family != 'Disproportionation' and \
#             reaction_list[index].family != 'intra_H_migration' and \
#             reaction_list[index].family != 'R_Addition_MultipleBond':
#             continue
        print(i,'\t', index, '\t', reaction_list[index].family, '\t', reaction_list[index], '\t')
    except AttributeError:
        continue
        print(i,'\t', index, '\t', 'PDEP', '\t', reaction_list[index], '\t')

Rank	Index	Family		Reaction
0 	 288 	 CurranPentane 	 OH(15) + butane(1) <=> H2O(8) + PC4H9(182) 	
1 	 247 	 CurranPentane 	 OH(15) + butane(1) <=> H2O(8) + SC4H9(183) 	
2 	 279 	 CurranPentane 	 SC4H9O2(186) <=> HO2(16) + C4H8-2(189) 	
3 	 323 	 CurranPentane 	 O2(2) + C4H8OOH1-3(219) <=> C4H8OOH1-3O2(225) 	
4 	 281 	 CurranPentane 	 O2(2) + C4H8OOH2-4(223) <=> C4H8OOH2-4O2(229) 	
5 	 316 	 CurranPentane 	 PC4H9O2(184) <=> HO2(16) + C4H8-1(188) 	
6 	 249 	 CurranPentane 	 HO2(16) + butane(1) <=> H2O2(17) + SC4H9(183) 	
7 	 449 	 CurranPentane 	 C4H8OOH1-3(219) <=> OH(15) + C4H8O1-3(214) 	
8 	 317 	 CurranPentane 	 SC4H9O2(186) <=> HO2(16) + C4H8-1(188) 	
9 	 326 	 CurranPentane 	 C4H8OOH1-3O2(225) <=> C4H71-1,3OOH(238) 	
10 	 290 	 CurranPentane 	 HO2(16) + butane(1) <=> H2O2(17) + PC4H9(182) 	
11 	 282 	 CurranPentane 	 C4H8OOH2-4O2(229) <=> C4H72-2,4OOH(244) 	
12 	 1701 	 CurranPentane 	 SC4H9O2(186) + butane(1) <=> SC4H9(183) + SC4H9O2H(187) 	
13 	 354 	 FFCM1(-) 	 HO2(16) + CH3CHO